# Read_WRF_towers
Use mmctools tslist class and tslist_seriesReader to read tower output for the FINO offshore microscale simulations.<br>
Save tslist dataseries to pdata file

## Set file paths, import 'sys' module for setting filepath

In [27]:
#wrf_dir = '/p/lscratchh/lassman1/a2e-mmc/WRF/FINO/ERA5_grid2.0/'
wrf_dir = '/p/lustre1/lassman1/a2e-mmc/FINO/WRF/resims/'
#cases = ['microscale_setup_v2_CONTROL', 'microscale_setup_v2_CPM']
cases = ['CONTROL', 'CPM']

pdata_save_dir = '/p/lscratchh/lassman1/a2e-mmc/pdata/FINO_RS/'


#restarts = ['tsout_01:00-01:20', 'tsout_01:20-01:40', 'tsout_01:40-02:00',\
#            'tsout_02:00-02:20', 'tsout_02:20-02:40', 'tsout_02:40-03:00',\
#            'tsout_03:00-03:20', 'tsout_03:20-03:40', 'tsout_03:40-04:00',\
#            'tsout_04:00-04:20', 'tsout_04:20-04:40', 'tsout_04:40-05:00']

restarts = ['tsout_0100-0120', 'tsout_0120-0140', 'tsout_0140-0200',\
            'tsout_0200-0220', 'tsout_0220-0240', 'tsout_0240-0300',\
            'tsout_0300-0320', 'tsout_0320-0340', 'tsout_0340-0400',\
            'tsout_0400-0420', 'tsout_0420-0440', 'tsout_0440-0500']

case_days_start = ['2010-05-16']
case_days_end = ['2010-05-16']

#wrf_start_times = ['2010-05-16 01:00', '2010-05-16 01:20', '2010-05-16 01:40', \
#                   '2010-05-16 02:00', '2010-05-16 02:20', '2010-05-16 02:40', \
#                   '2010-05-16 03:00', '2010-05-16 03:20', '2010-05-16 03:40', \
#                   '2010-05-16 04:00', '2010-05-16 04:20', '2010-05-16 04:40' ]

wrf_start_times = ['2010-05-14 12:00', '2010-05-14 12:00', '2010-05-14 12:00', \
                   '2010-05-14 12:00', '2010-05-14 12:00', '2010-05-14 12:00', \
                   '2010-05-14 12:00', '2010-05-14 12:00', '2010-05-14 12:00', \
                   '2010-05-14 12:00', '2010-05-14 12:00', '2010-05-14 12:00']



wrf_dt = 0.025 #[seconds]
import sys
import os

#module_path = os.path.join(os.environ['HOME'],'code/Python/a2e-mmc/')
module_path = os.path.join(os.environ['HOME'],'mmc/mmc_github_clones/mmctools/')                                                          
if module_path not in sys.path:
    sys.path.append(module_path)


In [16]:
tower_dict = { 
        "ARR01": (121,125, 0, 0),
        "ARR02": (121,225, 0, 1),
        "ARR03": (121,325, 0, 2),
        "ARR04": (121,425, 0, 3),
        "ARR05": (221,525, 0, 4),
        "ARR06": (221,125, 1, 0),
        "ARR07": (221,225, 1, 1),
        "ARR08": (221,325, 1, 2),
        "ARR09": (221,425, 1, 3),
        "ARR10": (321,525, 1, 4),
        "ARR11": (321,125, 2, 0),
        "ARR12": (321,225, 2, 1),
        "ARR13": (321,325, 2, 2),
        "ARR14": (321,425, 2, 3),
        "ARR15": (421,525, 2, 4),
        "ARR16": (421,125, 3, 0),
        "ARR17": (421,225, 3, 1),
        "ARR18": (421,325, 3, 2),
        "ARR19": (421,425, 3, 3),
        "ARR20": (521,525, 3, 4),
        "ARR21": (521,125, 4, 0),
        "ARR22": (521,225, 4, 1),
        "ARR23": (521,325, 4, 2),
        "ARR24": (521,425, 4, 3),
        "ARR25": (121,525, 4, 4),
             }

In [17]:
print(tower_dict["ARR01"])

(121, 125, 0, 0)


In [18]:
#ts_heights = [5.0, 10.0,  15.0, 20.0, 25.0, 30.0, 33.0, 35.0, 40.0, 45.0, 50.0, \
#                  55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 110.0, \
#                  125.0, 150.0, 200.0],

ts_heights = [40.0, 60.0, 80.0],

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from matplotlib.colors import Normalize

from mmctools.wrf.utils import Tower, combine_towers, tsout_seriesReader

In [20]:
n_case_days = len(case_days_start)
ncases = len(cases)

In [21]:
# Where the tslist output are located:
twr_file_loc = '/p/lscratchh/lassman1/a2e-mmc/WRF/FINO/ERA5_grid2.0/microscale_setup_v2_CONTROL/tsout_01:00-01:20/'
# Which tower you want to read in, and which domain:
twr_file_name = 'ARR09.d06' # 'FN1' is the name, 'd03' is for the third domain
twr_file_path = '{}{}'.format(twr_file_loc,twr_file_name)

In [22]:
single_tower = Tower(twr_file_path)

type(single_tower)

mmctools.wrf.utils.Tower

In [23]:
print(single_tower.time)

[37.000008 37.000011 37.000019 ... 37.398598 37.398605 37.398613]


In [24]:
# Create combined tower DataArray (this may take a while)
#twrs_combined_CONTROL = tsout_seriesReader(fdir=wrf_dir + cases[0]+'/',
#                                       restarts=restarts,
#                                       simulation_start_time=wrf_start_times,
#                                       structure='unordered',
#                                       time_step=wrf_dt,
#                                       domain_of_interest='d06',
#                                       heights=[5.0, 10.0,  15.0, 20.0, \
#                                                25.0, 30.0, 33.0, 35.0, \
#                                                40.0, 45.0, 50.0, 55.0, \
#                                                60.0, 65.0, 70.0, 75.0, \
#                                                80.0, 85.0, 90.0, 95.0, \
#                                                100.0, 110.0, 125.0, 150.0, 200.0],
#                                       height_var='ph',
#                                       select_tower = ['ARR06', 'ARR07', 'ARR08', 'ARR09']
#                                       )
twrs_combined_CONTROL = tsout_seriesReader(fdir=wrf_dir + cases[0]+'/',
                                       restarts=restarts[:],
                                       simulation_start_time=wrf_start_times[:],
                                       structure='unordered',
                                       time_step=wrf_dt,
                                       domain_of_interest='d06',
                                       heights=[40.0, 60.0, 80.0],                                       
                                       height_var='ph',
                                       select_tower = [
                                                       #'ARR01', 'ARR02', 'ARR03', 'ARR04', 'ARR05', \
                                                       #'ARR06', 'ARR07', 'ARR08', 'ARR09', 'ARR10', \
                                                       #'ARR11', 'ARR12', 'ARR13', 'ARR14', 'ARR15', \
                                                       #'ARR16', ' ARR17', 'ARR18', 'ARR19', 'ARR20', \
                                                       #'ARR21', 'ARR22', 'ARR23', 'ARR24', 'ARR25'
                                                       'ARR04', 'ARR05', 'ARR09', 'ARR10'
                                                       ]
                                       )
#33.0 40.0 50.0 60.0 70.0 80.0 90.0 100.0

restart: tsout_0100-0120
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0120-0140
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0140-0200
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0200-0220
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0220-0240
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0240-0300
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0300-0320
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0320-0340
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0340-0400
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0400-0420
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting 

In [25]:
# Create combined tower DataArray (this may take a while)
#dinds = [0,4]
#±restarts_cpm = np.delete( restarts, dinds )
#print(restarts_cpm)

In [26]:
# Create combined tower DataArray (this may take a while)
print(wrf_dir + cases[1]+'/')
twrs_combined_CPM = tsout_seriesReader(fdir=wrf_dir + cases[1]+'/',
                                       restarts=restarts,#_cpm,
                                       simulation_start_time=wrf_start_times[:],
                                       structure='unordered',
                                       #time_step=wrf_dt,
                                       domain_of_interest='d06',
                                       heights=[40.0, 60.0, 80.0], 
                                       height_var='ph',
                                       select_tower = [
                                                       #'ARR01', 'ARR02', 'ARR03', 'ARR04', 'ARR05', \
                                                       #'ARR06', 'ARR07', 'ARR08', 'ARR09', 'ARR10', \
                                                       #'ARR11', 'ARR12', 'ARR13', 'ARR14', 'ARR15', \
                                                       #'ARR16', ' ARR17', 'ARR18', 'ARR19', 'ARR20', \
                                                       #'ARR21', 'ARR22', 'ARR23', 'ARR24', 'ARR25'
                                                       'ARR04', 'ARR05', 'ARR09', 'ARR10'
                                                       ]
                                       )

/p/lustre1/lassman1/a2e-mmc/FINO/WRF/resims/CPM/
restart: tsout_0100-0120
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0120-0140
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0140-0200
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0200-0220
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0220-0240
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0240-0300
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0300-0320
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0320-0340
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0340-0400
starting ARR04.d06
starting ARR05.d06
starting ARR09.d06
starting ARR10.d06
restart: tsout_0400-0420
starting ARR04.d0

In [28]:
print(twrs_combined_CPM.datetime)
print(twrs_combined_CONTROL.datetime)

<xarray.DataArray 'datetime' (datetime: 579000)>
array(['2010-05-16T01:00:00.029000000', '2010-05-16T01:00:00.040000000',
       '2010-05-16T01:00:00.068000000', ..., '2010-05-16T05:01:14.956000000',
       '2010-05-16T05:01:14.981000000', '2010-05-16T05:01:14.995000000'],
      dtype='datetime64[ns]')
Coordinates:
  * datetime  (datetime) datetime64[ns] 2010-05-16T01:00:00.029000 ... 2010-0...
<xarray.DataArray 'datetime' (datetime: 579000)>
array(['2010-05-16T01:00:00.025000000', '2010-05-16T01:00:00.050000000',
       '2010-05-16T01:00:00.075000000', ..., '2010-05-16T05:01:14.950000000',
       '2010-05-16T05:01:14.975000000', '2010-05-16T05:01:15.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * datetime  (datetime) datetime64[ns] 2010-05-16T01:00:00.025000 ... 2010-0...


In [29]:
print( twrs_combined_CPM.variables)

Frozen({'theta': <xarray.Variable (datetime: 579000, nz: 3, station: 4)>
array([[[280.68015314, 280.73389492, 280.71499746, 280.79918762],
        [280.77288659, 280.72577183, 280.71304105, 280.74480335],
        [280.95500249, 280.83234367, 280.72736339, 280.72813701]],

       [[280.67989058, 280.73396416, 280.71377736, 280.79951471],
        [280.77241574, 280.72550722, 280.71210733, 280.74504245],
        [280.95474739, 280.83141592, 280.72657575, 280.72795466]],

       [[280.67959269, 280.73403041, 280.71258345, 280.79985334],
        [280.77196989, 280.72523073, 280.71116221, 280.74530422],
        [280.95451496, 280.83040922, 280.72576789, 280.72780109]],

       ...,

       [[281.11623744, 281.21412657, 281.15865   , 281.57248472],
        [281.19369676, 281.24738684, 281.11601604, 281.54345178],
        [281.69594404, 281.27560162, 281.2204585 , 281.34992244]],

       [[281.1170766 , 281.21474623, 281.15981424, 281.57242929],
        [281.19427243, 281.24699771, 281.1172354

In [30]:
twrs_combined_CPM_ds = twrs_combined_CPM.resample({'datetime' : '0.4S'}).nearest()
twrs_combined_CONTROL_ds = twrs_combined_CONTROL.resample({'datetime' : '0.4S'}).nearest()

/g/g12/lassman1/anaconda3/envs/wildfire/lib/python3.8/site-packages/xarray/core/common.py:1123: FutureWarning: 'base' in .resample() and in Grouper() is deprecated.
The new arguments that you should use are 'offset' or 'origin'.

>>> df.resample(freq="3s", base=2)

becomes:

>>> df.resample(freq="3s", offset="2s")

  grouper = pd.Grouper(


In [31]:
CPM_save = xr.Dataset(
                {
                    'u': twrs_combined_CPM_ds.u,
                    'v': twrs_combined_CPM_ds.v,
                    'w': twrs_combined_CPM_ds.w
                }
            )

CONTROL_save = xr.Dataset(
                {
                    'u': twrs_combined_CONTROL_ds.u,
                    'v': twrs_combined_CONTROL_ds.v,
                    'w': twrs_combined_CONTROL_ds.w
                }
            )


In [32]:
print(CPM_save.data_vars)
print(CPM_save.coords)
print(CPM_save.dims)

Data variables:
    u        (datetime, nz, station) float64 12.59 12.05 11.27 ... 12.27 14.66
    v        (datetime, nz, station) float64 -5.177 -7.06 ... -2.453 -4.254
    w        (datetime, nz, station) float64 0.09826 -0.1161 ... -0.406 -1.223
Coordinates:
  * datetime  (datetime) datetime64[ns] 2010-05-16T01:00:00 ... 2010-05-16T05...
    lat       (station) float64 54.01 54.01 54.01 54.01
    lon       (station) float64 6.587 6.602 6.588 6.603
    zsurface  (station) float64 0.0 0.0 0.0 0.0
    height    (nz) float64 40.0 60.0 80.0
  * station   (station) object 'ARR04' 'ARR05' 'ARR09' 'ARR10'
Frozen(SortedKeysDict({'datetime': 36188, 'station': 4, 'nz': 3}))


### Save to process data files

In [33]:
print(pdata_save_dir)

/p/lscratchh/lassman1/a2e-mmc/pdata/FINO_RS/


In [34]:
CPM_save.to_netcdf( pdata_save_dir + 'FINO_wrf-tslist-processed_cpm.nc')

In [35]:
CONTROL_save.to_netcdf( pdata_save_dir + 'FINO_wrf-tslist-processed_control.nc')

### Some calculations and a plot to verify things are working correctly

In [36]:


print(twrs_combined_CONTROL.coords)
print( type(twrs_combined_CONTROL.datetime.values[0]), twrs_combined_CONTROL.datetime.values[0])


Coordinates:
    lat       (station) float64 54.01 54.01 54.01 54.01
    lon       (station) float64 6.587 6.602 6.588 6.603
    zsurface  (station) float64 0.0 0.0 0.0 0.0
  * datetime  (datetime) datetime64[ns] 2010-05-16T01:00:00.025000 ... 2010-0...
    height    (nz) float64 40.0 60.0 80.0
  * station   (station) object 'ARR04' 'ARR05' 'ARR09' 'ARR10'
<class 'numpy.datetime64'> 2010-05-16T01:00:00.025000000


In [37]:


print(twrs_combined_CONTROL.coords)
print( type(twrs_combined_CONTROL.datetime.values[0]), twrs_combined_CONTROL.datetime.values[0])
print(np.datetime64('2010-05-17T14:38:00.025000000' ))

Coordinates:
    lat       (station) float64 54.01 54.01 54.01 54.01
    lon       (station) float64 6.587 6.602 6.588 6.603
    zsurface  (station) float64 0.0 0.0 0.0 0.0
  * datetime  (datetime) datetime64[ns] 2010-05-16T01:00:00.025000 ... 2010-0...
    height    (nz) float64 40.0 60.0 80.0
  * station   (station) object 'ARR04' 'ARR05' 'ARR09' 'ARR10'
<class 'numpy.datetime64'> 2010-05-16T01:00:00.025000000
2010-05-17T14:38:00.025000000


In [38]:
twrs_combined_CONTROL['wspd'] = np.sqrt( twrs_combined_CONTROL['u']**2. + twrs_combined_CONTROL['v']**2. )
twrs_combined_CPM['wspd'] = np.sqrt( twrs_combined_CPM['u']**2. + twrs_combined_CPM['v']**2. )

In [39]:
print(twrs_combined_CONTROL.height.sel( station = 'ARR09').isel(datetime = -1).values )

ValueError: dimensions or multi-index levels ['station'] do not exist

In [ ]:
### Quick calculation and plot
t0 = pd.to_datetime( '2010-05-16T01:40:00' )
tf = pd.to_datetime( '2010-05-16T02:00:00' )
print( twrs_combined_CONTROL.datetime.values[-1],  twrs_combined_CONTROL.datetime.values[0])

#test = twrs_combined_CONTROL.u.sel( station = 'ARR09', datetime = slice(t0, tf) )#.mean(dim = 'datetime')
#print(test.values)
plt.figure()
plt.plot( twrs_combined_CONTROL.wspd.sel( station = 'ARR09', \
                            datetime = slice(t0, tf) ).mean(dim = 'datetime'), \
          twrs_combined_CONTROL.height.sel( station = 'ARR09').isel(datetime = -1),
          label = 'CONTROL' )

plt.plot( twrs_combined_CPM.wspd.sel( station = 'ARR09', \
                            datetime = slice(t0, tf) ).mean(dim = 'datetime'), \
          twrs_combined_CPM.height.sel( station = 'ARR09').isel(datetime = -1),
          label = 'CPM' )
plt.xlabel('WSPD [[m/s]]')
plt.ylabel('Height [m]')
plt.legend(loc = 'best')